In [2]:
import pandas as pd ; 
import numpy as np 
import time , os , sys , random
import sqlite3 , re


def convert_db_data_to_csv(table_name , connection  , path = './data/' , filename="mydata.csv" ):
    df = pd.DataFrame(pd.read_sql('select * from '+table_name , connection)) ; 
    df.to_csv(os.path.join(path , filename)) ;
    

In [379]:
class allotment:
    def __init__(self , subjectid ,  facultyid1 , facultyid2 , roomid , dayno , hour , subjectname='' ):
        self.subjectid =subjectid ; 
        self.subjectname = subjectname
        self.facultyid1 = facultyid1
        self.facultyid2 = facultyid2
        self.roomid = roomid
        self.dayno = dayno
        self.hour = hour
        

class faculty:
    def __init__(self , fid , fname , ftype):
        self.id , self.name , self.type = fid , fname , ftype ; 
    def __str__(self):
        return "< ID : {} , Name : {}  , type : {}  >".format(self.id , self.name , self.type) ;
    def __repr__(self):
        return self.__str__() ;
        
        


class Timetable:
    
    def __init__(self):
        self.faculty_data = pd.read_csv("data/faculty.csv") ; 
        self.room_data = pd.read_csv('data/room.csv') ; 
        self.subject_data = pd.read_csv('data/subject.csv') ;
        self.test_data = pd.read_csv('data/test.csv') ;
        
        self.faculty_data = self.faculty_data[['facultyId' , 'facultyName' , 'facultyType']]
        self.room_data = self.room_data[['roomId' , 'roomType' , 'roomNo']]
        self.subject_data = self.subject_data[['subjectId' , 'subjectName' , 'subjectType' , 'credits']]
        self.test_data = self.test_data[['batchID' , 'subjectID' , 'facultyID1' , 'facultyID2']]
        
        self.sectionslist = self.get_sections_from_batches() ;  
        self.normal_rooms = self.room_data[(self.room_data.roomType==0) & (self.room_data.roomNo)]
        self.lab_rooms = self.room_data[(self.room_data.roomType==1) & (self.room_data.roomNo)]
    
        self.allotments = []
        self.section_to_subjects = {}
        self.populate_section_to_subject() ; 
        
        self.create_faculty_objects() ; 
        self.init_faculties() ;
    
   
    def populate_section_to_subject(self):
        for section in self.sectionslist:
            self.section_to_subjects[section] = self.test_data[self.test_data.batchID==section] ;
            
    
        
    def get_sections_from_batches(self):
        '''Assume the string till last but one char as the section name and if there is only 1 char take that as section name'''
        def myfun(x , compiledre):
            if(len(x)==1):
                return x ;
            else:
                if(compiledre.match(x)):
                    return x;  
        
        compiledre = re.compile('^.*?[a-zA-Z]$') ;
        sections = self.test_data.batchID.apply(lambda x : myfun(x , compiledre)) ;
        sectionslist = sections[sections.apply(lambda x : True if x else False)].unique()
        return sectionslist ;
        

        
    def init_faculties(self):
        '''Initialize the faculties to be not allocated to all the hours of all days'''        
        self.faculty_to_day_hour_slot_map = {}
        day_to_hour = {}
        faculty_to_data = {} 

        hour_to_allotflag = {}
        for hour in range(8):
            hour_to_allotflag.update({hour : {'alloted ': False}}) ;

        for day in ['mon' , 'tue' , 'wed' , 'thu' , 'fri' , 'sat']:
            day_to_hour.update({day : hour_to_allotflag}) ;

        for faculty_object in self.faculties:
            self.faculty_to_day_hour_slot_map.update({faculty_object : day_to_hour}) ;
            
                    
    
    def allot_slots_normal_class(self):
        
        allotment = pd.DataFrame(columns=['day' ,'section', 'hour', 'subjectid' , 'roomno' , 'facultyid']) ;
        
        for section in self.sectionslist:
            selected_room = self.normal_rooms.roomNo.sample().values[0] ;
        
            for day in ['mon' , 'tue' , 'wed' , 'thu' , 'fri' , 'sat']:
                batch_subs = self.section_to_subjects[section].subjectID.values ;
                random.shuffle(batch_subs) ; 

                for hour in range(1 , 9):
                    if(not batch_subs.any()):
                        break ; 
                    
                    selected_subject = batch_subs[0] ;
                    
                    batch_subs = np.delete(batch_subs , 0) ;
                    selected_faculty = self.test_data[self.test_data['subjectID']==selected_subject].facultyID1.sample().values[0] ;
                    allotment = allotment.append({'section' : section , 'day' : day , 'hour' : hour  , 'subjectid' : selected_subject , 'roomno' : selected_room , 'facultyid' : selected_faculty  } , ignore_index=True)
            
        self.allotment = allotment ; 
                    
    
    
    def create_faculty_objects(self):
        '''Create facultie objects using the data ''' 
        self.faculties = set() ;
        for i in range(self.faculty_data.count().facultyId):
            self.faculties.add(faculty(self.faculty_data.loc[i].facultyId , self.faculty_data.loc[i].facultyName , self.faculty_data.loc[i].facultyType))
                
        
                    
    
    def check_allotment_validity(self , allotment ):
        for allots in self.allotments:
            pass ; 
            


In [380]:
obj = Timetable()
obj.allot_slots_normal_class()

In [349]:
a  = obj.test_data.subjectID.values
# a = np.delete(a , 2)
# a.item('4CCI03')


In [381]:
obj.allotment

,day,section,hour,subjectid,roomno,facultyid
0,mon,4A,1,MC03,CSL003,48
1,mon,4A,2,FR,CSL003,42
2,mon,4A,3,4CCI04,CSL003,41
3,mon,4A,4,4CCI02,CSL003,21
4,mon,4A,5,4MAT2,CSL003,49
5,mon,4A,6,4CCI03,CSL003,36
6,mon,4A,7,4CS01,CSL003,39
7,mon,4A,8,4CCI01,CSL003,29
8,tue,4A,1,4CCI03,CSL003,21
9,tue,4A,2,FR,CSL003,42
